In [1]:
import numpy as np
import mediapipe as mp
import cv2

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [2]:
class Excersice:
    def __init__(self, s1, s2, s3):
        self.s1 = s1
        self.s2 = s2
        self.s3 = s3
    def feedback(self, pose):
        if pose == self.s1:
            return 0
        elif pose == self.s2:
            return 1
        elif pose == self.s3:
            return 2
        else:
            return -1
        

In [3]:
def calculate_angle(a, b, c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End

    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)

    if angle > 180.0:
        angle = 360 - angle
    
    return angle

- squat 기준으로 피드백 발생기 만들기
- class로 만들어서 모든 운동 기준으로 바꾸기
- 접근이 용이하게 칼럼 선택하기

## 초기조건
- 카메라는 왼쪽을 기준으로
- 카메라는 화면에 수직으로 
  

In [4]:
# Excersice data


# left side
knee = [mp_pose.PoseLandmark.LEFT_HIP.value, mp_pose.PoseLandmark.LEFT_KNEE.value, mp_pose.PoseLandmark.LEFT_ANKLE.value]
hip = [mp_pose.PoseLandmark.LEFT_SHOULDER.value, mp_pose.PoseLandmark.LEFT_HIP.value, mp_pose.PoseLandmark.LEFT_KNEE.value]
ankle = [mp_pose.PoseLandmark.LEFT_KNEE.value, mp_pose.PoseLandmark.LEFT_ANKLE.value, mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value]
elbow = [mp_pose.PoseLandmark.LEFT_SHOULDER, mp_pose.PoseLandmark.LEFT_ELBOW, mp_pose.PoseLandmark.LEFT_WRIST]
shoulder = [mp_pose.PoseLandmark.LEFT_HIP, mp_pose.PoseLandmark.LEFT_SHOULDER, mp_pose.PoseLandmark.LEFT_ELBOW]
wrist = [mp_pose.PoseLandmark.LEFT_ELBOW, mp_pose.PoseLandmark.LEFT_WRIST, mp_pose.PoseLandmark.LEFT_INDEX]

squat_point = [knee, hip, ankle]

pushup_point = [elbow, shoulder, wrist, hip, knee]

left_armcurl_point = [elbow]

lunge_point = [knee, hip, ankle]

pullup_point = [elbow, shoulder, wrist, hip, knee]

# landmark에 연결된 부위 찾기
print(mp_pose.PoseLandmark.LEFT_HIP)

PoseLandmark.LEFT_HIP


In [5]:
## angle feedback function
def angle_feedback(angle_dict, state, limit_params, comments):
    fb_comments = list()
    if state == 'Squat':
        points = angle_dict.keys()
        for i, point in enumerate(points):
            comment = feedback(point, angle_dict[point], limit_params[i], comments[i])
            fb_comments.append(comment)
    elif state == 'Pushup':
        pass
    return fb_comments



def feedback(point, angle, limit, comment):
    print(point)
    if angle > limit:
        return comment
    else:
        return 'Good'

In [6]:
# parameter of excerise

# squart
squart_limit_params = [27, 84, 23]
squart_comments = ['Knee is too much bent', 'Hip is too much bent', 'Ankle is too much bent']

In [8]:
state = 'Squat'

cap = cv2.VideoCapture('/Users/iyongbin/Repository/Lenfit/ML/Dataset/video/bodysquat05.mp4')

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        # read video
        ret, frame = cap.read()
        if not ret:
            break
        # frame size
        height, width, _ = frame.shape
        print(width, height)

        # color change
        img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img.flags.writeable = False

        # detection
        results = pose.process(img)

        # draw landmarks
        img.flags.writeable = True
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

        # exception 
        try:
            landmarks = results.pose_landmarks.landmark
            angle = dict()

            # calculate angle
            if state == 'Squat':
                for points in squat_point:
                    # get coordinates
                    xy_data = [[landmarks[point].x, landmarks[point].y] for point in points]

                    # calculate angle
                    # x_b, y_b = np.multiply(xy_data[1], [width, height]) - [0, 300]
                    angle[points[1]] = 90 - calculate_angle(xy_data[0], xy_data[1], xy_data[1]+[0, -1]) # vertical angel

                    # feedback
                    fb = angle_feedback(angle, state, squart_limit_params, squart_comments)
                    print(np.multiply(xy_data[1], [width, height]).astype(int))
                    # visualize angle
                    cv2.putText(img, str(angle[points[1]]),
                                tuple(np.multiply(xy_data[1], [width, height]).astype(int)), # np.multiply(xy_data[1], [640, 480]).astype(int)의 의미: x, y 좌표를 640, 480으로 곱해준다.
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA
                                )
                    
                    
                    # print(x_b, y_b)


            elif state == 'Pushup':
                pass
            elif state == 'Lunge':
                pass
            elif state == 'Pullup':
                pass
            elif state == 'ArmCurl':
                pass
            # Visualize angle

        except:
            print('error')
        print(fb)


        mp_drawing.draw_landmarks(img, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                    mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                    mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
                                    )
        
        cv2.imshow('VideoFrame', img)
        if cv2.waitKey(1) == ord('q'):
            break



# 동영상 재생 종료
cap.release()
# 화면에 나타난 윈도우들을 종료
cv2.destroyAllWindows()


1280 720
25
[909 488]
25
23
[901 336]
25
23
27
[925 639]
['Knee is too much bent', 'Hip is too much bent', 'Ankle is too much bent']
1280 720
25
[909 487]
25
23
[902 336]
25
23
27
[925 638]
['Knee is too much bent', 'Hip is too much bent', 'Ankle is too much bent']
1280 720
25
[909 486]
25
23
[902 335]
25
23
27
[926 638]
['Knee is too much bent', 'Hip is too much bent', 'Ankle is too much bent']
1280 720
25
[909 486]
25
23
[902 335]
25
23
27
[926 638]
['Knee is too much bent', 'Hip is too much bent', 'Ankle is too much bent']
1280 720
25
[909 486]
25
23
[903 335]
25
23
27
[926 638]
['Knee is too much bent', 'Hip is too much bent', 'Ankle is too much bent']
1280 720
25
[909 486]
25
23
[903 335]
25
23
27
[926 637]
['Knee is too much bent', 'Hip is too much bent', 'Ankle is too much bent']
1280 720
25
[909 486]
25
23
[903 335]
25
23
27
[926 637]
['Knee is too much bent', 'Hip is too much bent', 'Ankle is too much bent']
1280 720
25
[909 486]
25
23
[903 335]
25
23
27
[926 637]
['Knee is to

In [83]:
import cv2
import mediapipe as mp

# Mediapipe Pose Estimation 초기화
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

# OpenCV 비디오 캡처 초기화
cap = cv2.VideoCapture(0)

while True:
    # 비디오 프레임 읽기
    ret, frame = cap.read()
    
    # 이미지를 BGR에서 RGB로 변환
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # 이미지에서 포즈 추정
    results = pose.process(image)
    
    # 포즈 추정 결과에서 각 포인트 추출
    landmarks = results.pose_landmarks
    
    if landmarks is not None:
        # 각도 계산을 위한 포인트 인덱스
        point_a = landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER]
        point_b = landmarks.landmark[mp_pose.PoseLandmark.LEFT_ELBOW]
        point_c = landmarks.landmark[mp_pose.PoseLandmark.LEFT_WRIST]
        
        # 포인트의 좌표를 이미지 상의 위치로 변환
        height, width, _ = frame.shape
        x_a, y_a = int(point_a.x * width), int(point_a.y * height)
        x_b, y_b = int(point_b.x * width), int(point_b.y * height)
        x_c, y_c = int(point_c.x * width), int(point_c.y * height)
        
        # 두 벡터 생성
        vector_ab = [x_b - x_a, y_b - y_a]
        vector_bc = [x_c - x_b, y_c - y_b]
        
        # 두 벡터 사이의 각도 계산
        angle = cv2.fastAtan2(vector_bc[1], vector_bc[0]) - cv2.fastAtan2(vector_ab[1], vector_ab[0])
        
        # 각도를 화면에 표시
        cv2.putText(frame, f"Angle: {angle:.2f}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        
        # 포인트와 선 그리기
        cv2.circle(frame, (x_a, y_a), 5, (0, 255, 0), -1)
        cv2.circle(frame, (x_b, y_b), 5, (0, 255, 0), -1)
        cv2.circle(frame, (x_c, y_c), 5, (0, 255, 0), -1)
        cv2.line(frame, (x_a, y_a), (x_b, y_b), (255, 0, 0), 2)
        cv2.line(frame, (x_b, y_b), (x_c, y_c), (255, 0, 0), 2)
        
        # 추가적인 선 그리기
        cv2.line(frame, (x_a, y_a), (x_c, y_c), (0, 0, 255), 2)
    
    # 화면에 출력
    cv2.imshow("Pose Estimation", frame)
    
    # 'q' 키를 누르면 종료
